<a id="top"></a>
<div class="list-group" id="list-tab" role="tablist">
<h1 class="list-group-item list-group-item-action active" data-toggle="list" style='background:#005097; border:0' role="tab" aria-controls="home"><center>Hyperparameter Optimization</center></h1>

# Hyperparameter optimization

In machine learning, a **hyperparameter** is a value that controls the learning process of a model, but it's not directly learned from the data itself. Hyperparameters are essentially dials or knobs that must be adjusted before running the learning algorithm. They define the learning algorithm's overall behavior and influence the resulting model's complexity and capacity. In contrast to the model **parameters** that are learned during training, the hyperparameters are set before the training of a model begins. 

Some examples of hyperparameters are given below.

* the learning rate in logistic regression and in linear regression,
* the height of trees in decision tree learning,
* the value of $k$ in $k$-NN,
* the value of $k$ in $k$-means,
* the values of $\epsilon$ and $\operatorname{minPoints}$ in DBSCAN,
* the regularization term in linear regression,
* the polynomial degree in polynomial regression.


Hyperparameters can be considered as configurations of a learning algorithm. In general, the ideal settings for the algorithm to generate a suitable model for one dataset may not be the same for another dataset.

*Hyperparameter optimization* (aka *hyperparameter tuning*) is the procedure of defining appropriate values for the hyperparameters of a given learning algorithm. The general approach is to generate several combinations of hyperparameter values. Then, a model is trained for each combinatation. Finally, the best combination (according to some evaluation measure applied on the candidate models) is chosen.

When the learning algorithm generates an estimator (either a classifier or a regressor), there are two main strategies to *automatically* tune its hyperparameters: *Grid Search* and *Random Search*. Both strategies generate several hyperparameter combinations. Then, for each combination, an estimator is trained a model (on the training set) and evaluateed (on the validation set). The difference between these two strategies is in the way the several combinations are generated, as described below.

**Grid search** first defines a grid of hyperparameter combinations. The amount of hyperparameters defines the number of dimensions in the grid. Then, each combination (a point in the grid) ​​is used to produce a model.

The image below ([source](https://www.researchgate.net/publication/271771573_TuPAQ_An_Efficient_Planner_for_Large-scale_Predictive_Analytic_Queries/figures?lo=1)) illustrates the GS procedure for a learning algorithm with two hyperparameters. The star represents the combination of hyperparameters that gives the best model. as measured in the validation set. The red dot in picture on the left represents the first combination test by GS. The picture on the center represents the second combination. The picture on the right illustrates all the tested combinations in the 2-dimensional grid.

<p align="center">
<img src="https://www.researchgate.net/profile/Michael_Jordan13/publication/271771573/figure/fig4/AS:668513593217027@1536397469229/Illustration-of-naive-grid-search-for-a-single-model-family-with-two-hyperparameters_W640.jpg">
</p>

**Random search** selects random combinations of hyperparameter values ​​(within preconfigured ranges of values for each hyperparameter) to train and evaluate the candidate models. Instead of trying all possible combinations of values, RS performs a pre-defined number of iterations, testing a random combination of hyperparameters ​​in each iteration.

Both GS and RS can be very computationally expensive. For example, searching for 20 different values ​​for each of 4 hyperparameters would require 160,000 k-fold cross-validation runs. If $k = 10$ then 1,600,000 model adjustments and 1,600,000 evaluations should be performed

The image below ([source](https://community.alteryx.com/t5/Data-Science/Hyperparameter-Tuning-Black-Magic/ba-p/449289)) illustrates the difference between GS and RS. Think of a learning algorithm with just two hyperparameters. This way, each combination of its hyperparameters is a pair of numbers. Suppose one of these hyperparameters (x-axis) has more influence than the other (y-axis) on the predictive performance of the generated models. The plot on the left shows several of these pairs organized in a grid. In total, there are thirty combination in this grid. The picture on the right shows other combinations of pairs; this times these combinations where randomly selected. Notice that RS has the potential to explore more promising combinations than GS.

<p align="center">
<img src="https://pvsmt99345.i.lithium.com/t5/image/serverpage/image-id/74545i97245FDAA10376E9/image-size/large?v=1.0&px=999">
</p>

In Scikit-Learn, the classes [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) and [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) implement GS and RS, respectively.

### Class GridSearchCV

Scikit-Learn provides the class `GridSearchCV` to perform grid search.

The following code block presents an example of using the `GridSearchCV` class to find the ideal polynomial model for a data set. In this example, a three-dimensional grid of hyperparameters is explored:

* the polynomial degree,
* a boolean indicator that indicates whether the linear coefficient should be adjusted,
* an indicator (boolean) indicating whether the data should be normalized.

In [1]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# Load dataset
iris = load_iris()
X, y = iris.data, iris.target

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Define the model
svc = SVC()

# Define the parameter grid to search
param_grid = {
    'C': [0.1, 1, 10],
    'kernel': ['linear', 'rbf'],
    'gamma': ['scale', 'auto']
}

# Set up GridSearchCV
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy')

# Fit the model
grid_search.fit(X_train, y_train)

# Print the best parameters and the best score
print("Best parameters found:", grid_search.best_params_)
print("Best cross-validation score: {:.2f}".format(grid_search.best_score_))

# Evaluate on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("\nClassification Report on Test Set:\n", classification_report(y_test, y_pred))


Best parameters found: {'C': 1, 'gamma': 'scale', 'kernel': 'linear'}
Best cross-validation score: 0.96

Classification Report on Test Set:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00        19
           1       1.00      1.00      1.00        13
           2       1.00      1.00      1.00        13

    accuracy                           1.00        45
   macro avg       1.00      1.00      1.00        45
weighted avg       1.00      1.00      1.00        45



### Class RandomizedSearchCV

This allows you to explicitly control the number of parameter combinations that are attempted. The number of search iterations is defined based on time or resources. Scikit Learn offers the RandomizedSearchCV function for this process.

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from timeit import default_timer as timer
from sklearn.svm import LinearSVC

def linear_SVC(x, y, param, kfold):
    param_grid = {'C':param}
    k = StratifiedKFold(n_splits=kfold)
    grid = GridSearchCV(LinearSVC(dual=False), param_grid=param_grid, cv=k, n_jobs=4, verbose=1)

    return grid.fit(x, y)

def Linear_SVC_Rand(x, y, param, kfold, n):
    param_grid = {'C':param}
    k = StratifiedKFold(n_splits=kfold)
    randsearch = RandomizedSearchCV(LinearSVC(dual=False), param_distributions=param_grid, cv=k, n_jobs=4,
                                    verbose=1, n_iter=n)

    return randsearch.fit(x, y)

from sklearn.datasets import load_iris
iris = load_iris()
X = iris.data
y = iris.target

x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


start = timer()
param = [i/1000 for i in range(1,1000)]
param1 = [i for i in range(1,101)]
param.extend(param1)

#progress = progressbar.bar.ProgressBar()
clf = Linear_SVC_Rand(x=x_train, y=y_train, param=param, kfold=3, n=100)

print('LinearSVC:')
print('Best cv accuracy: {}' .format(clf.best_score_))
print('Test set score:   {}' .format(clf.score(x_test, y_test)))
print('Best parameters:  {}' .format(clf.best_params_))
print()

duration = timer() - start
print('time to run RandomizedSearchCV: {}' .format(duration))


print('########')

#high C means more chance of overfitting

start = timer()
param = [i/1000 for i in range(1,1000)]
param1 = [i for i in range(1,101)]
param.extend(param1)

clf = linear_SVC(x=x_train, y=y_train, param=param, kfold=3)

print('LinearSVC:')
print('Best cv accuracy: {}' .format(clf.best_score_))
print('Test set score:   {}' .format(clf.score(x_test, y_test)))
print('Best parameters:  {}' .format(clf.best_params_))
print()

duration = timer() - start
print('time to run GridSearchCV: {}' .format(duration))

Fitting 3 folds for each of 100 candidates, totalling 300 fits
LinearSVC:
Best cv accuracy: 0.9666666666666667
Test set score:   0.9666666666666667
Best parameters:  {'C': 0.703}

time to run RandomizedSearchCV: 2.061651249998249
########
Fitting 3 folds for each of 1099 candidates, totalling 3297 fits
LinearSVC:
Best cv accuracy: 0.9666666666666667
Test set score:   0.9666666666666667
Best parameters:  {'C': 0.441}

time to run GridSearchCV: 0.7237497499445453


# Optuna

Optuna is an open-source hyperparameter optimization framework that automates the process of tuning machine learning models by efficiently searching for the best hyperparameters. It supports both classical machine learning models and deep learning models, and its goal is to maximize or minimize a given **objective function**, such as model performance, by optimizing hyperparameters.

In the Optuna optimization framework, a **study** is the main object that represents an optimization session. It encompasses the entire optimization process — including the definition of the objective function, the trials (each set of hyperparameters tested), and the history and results of those trials.

Optuna selects the **sampling strategy** based on the **study configuration** and **hyperparameter types**. The choice depends on the search space, the number of trials, and whether prior knowledge is available.

## Default Strategy: Tree-structured Parzen Estimator (TPE)
- If no specific sampler is specified, Optuna **automatically** uses **TPE (Tree-structured Parzen Estimator)**.
- TPE **models the probability distribution** of good and bad hyperparameter choices and chooses new trials accordingly.
- Best for **non-convex search spaces** where grid/random search fails.

Example of instantiating a study that uses TPE:

```python
import optuna
study = optuna.create_study(direction="maximize")  # Uses TPE by default

## Explicitly Specifying a Sampler
You can override the default and choose a specific sampler:

| Sampler                  | When to Use?                                           | Example                                      |
|--------------------------|------------------------------------------------------|----------------------------------------------|
| **TPE (default)**        | Works well in most cases, adaptive Bayesian optimization | `optuna.samplers.TPESampler()`               |
| **Random Search**        | Good for benchmarking, large search spaces          | `optuna.samplers.RandomSampler()`           |
| **Grid Search**          | If you have limited trials and want exhaustive search | `optuna.samplers.GridSampler(search_space)` |
| **CMA-ES**              | Good for continuous spaces, often used in reinforcement learning | `optuna.samplers.CmaEsSampler()` |

Example of choosing a sampler:

```python
import optuna

sampler = optuna.samplers.RandomSampler()  # Choose random search
study = optuna.create_study(direction="maximize", sampler=sampler)

## How Optuna Adapts to the Problem
Optuna adjusts the search strategy based on:

- Discrete vs. Continuous Parameters. If parameters are categorical (trial.suggest_categorical), TPE handles them well. If parameters are continuous (trial.suggest_float), CMA-ES or TPE works better.
- Log-scaled vs. Linear Search Space. If log=True is used (e.g., learning_rate), Optuna adjusts the sampling distribution accordingly.
- Early Pruning and Convergence. If trials are pruned early, TPE focuses on exploiting promising areas rather than random exploration.

## Customizing the Sampling Strategy
You can combine samplers or switch strategies mid-experiment:

```python
sampler = optuna.samplers.TPESampler(n_startup_trials=10)  # Use random search for first 10 trials
study = optuna.create_study(sampler=sampler)


## Demo

In [1]:
# pip install optuna

In [8]:
import pandas as pd

data = pd.read_csv('../data/aug_train.csv')

# Split features and target
X = data.drop(columns=['id', 'Response'])
y = data['Response']

# Define categorical and numerical features
categorical_features = ['Gender', 'Vehicle_Age', 'Vehicle_Damage']
numerical_features = X.columns.difference(categorical_features)

In [10]:
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score
import optuna

# Preprocessing pipeline
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(), categorical_features)
])

N_TRIALS = 10
outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
outer_scores = []

for train_idx, test_idx in outer_cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    def objective(trial):
        n_estimators = trial.suggest_int('n_estimators', 50, 300)
        max_depth = trial.suggest_int('max_depth', 3, 20)
        min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
        min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)

        model = Pipeline([
            ('preprocessor', preprocessor),
            ('classifier', RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_split=min_samples_split,
                min_samples_leaf=min_samples_leaf,
                random_state=42
            ))
        ])

        inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
        scores = cross_val_score(model, X_train, y_train, cv=inner_cv, scoring='f1', n_jobs=-1)
        return np.mean(scores)

    # Run Optuna for current outer fold
    study = optuna.create_study(direction='maximize')
    study.optimize(objective, n_trials=N_TRIALS)

    # Train model with best params on full inner training set
    best_params = study.best_params
    final_model = Pipeline([
        ('preprocessor', preprocessor),
        ('classifier', RandomForestClassifier(**best_params, random_state=42))
    ])
    final_model.fit(X_train, y_train)
    y_pred = final_model.predict(X_test)
    score = f1_score(y_test, y_pred)
    outer_scores.append(score)

print("Nested CV F1 scores:", outer_scores)
print("Mean F1 score:", np.mean(outer_scores))


[I 2025-04-10 05:27:13,275] A new study created in memory with name: no-name-74b16ce9-45fa-4e35-ace1-193434f61b06


[I 2025-04-10 05:27:26,358] Trial 0 finished with value: 0.3414115867516296 and parameters: {'n_estimators': 69, 'max_depth': 14, 'min_samples_split': 5, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:27:58,037] Trial 1 finished with value: 0.00027945307348764586 and parameters: {'n_estimators': 239, 'max_depth': 7, 'min_samples_split': 3, 'min_samples_leaf': 8}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:28:30,901] Trial 2 finished with value: 0.27134730746322805 and parameters: {'n_estimators': 191, 'max_depth': 13, 'min_samples_split': 2, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:29:12,136] Trial 3 finished with value: 0.06669601151876058 and parameters: {'n_estimators': 285, 'max_depth': 9, 'min_samples_split': 5, 'min_samples_leaf': 6}. Best is trial 0 with value: 0.3414115867516296.
[I 2025-04-10 05:29:55,640] Trial 4 finished with value: 0.07699580221032808 and paramete

Nested CV F1 scores: [0.33155859440677016, 0.414046590175334, 0.4091045823724055, 0.3943604413567634, 0.4289385637395141]
Mean F1 score: 0.39560175441015744


In [11]:
%%time

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

# Train final model with best hyperparameters
best_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(
        n_estimators=best_params['n_estimators'],
        # max_depth=best_params['max_depth'],
        min_samples_split=best_params['min_samples_split'],
        min_samples_leaf=best_params['min_samples_leaf'],
        random_state=42
    ))
])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Train and evaluate the model
best_model.fit(X_train, y_train)
y_pred = best_model.predict(X_test)

# Evaluate performance
f1 = f1_score(y_test, y_pred)
print("F1-score on test set:", f1)

F1-score on test set: 0.4440074991997805
CPU times: user 48.3 s, sys: 75 ms, total: 48.4 s
Wall time: 47.3 s


In [6]:
# Train and evaluate the model
default_model = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(random_state=42))
])

default_model.fit(X_train, y_train)
y_pred = default_model.predict(X_test)
# Evaluate performance
f1 = f1_score(y_test, y_pred)
print("F1-score on test set:", f1)

F1-score on test set: 0.43990114580993034
